# Setting up the environment:
* This step clones the 'gobreak' repository from GitHub.
* This repository contains the necessary files and input data for the subsequent steps of the project.


In [ ]:
import os

if not os.path.exists('gobreak'):
    !git clone https://github.com/haroonsin/gobreak.git
else:
    print("gobreak repository already cloned.")


# Load the input dataset
* This step identifies the various time sequenced data folders used for processing.



In [ ]:
# Libraries and Dependencies
# ==========================
import cv2
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

# Function Definitions
# =====================

def get_subdirectories(directory):
    """Return a sorted list of subdirectories in the provided directory.

    Parameters:
    - directory (str): Path to the directory to search.

    Returns:
    - list: Sorted list of subdirectories.
    """
    return sorted([folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))])

# Data Initialization for Processing
# ==================================

# Define the directory containing the data.
data_dir = 'gobreak/data'

# Fetching the subdirectories
subdirectories = get_subdirectories(data_dir)
subdirectories

# ==================================================================
# **Image Analysis Functions**
### **Summary**:
###        This section contains functions that assist in analyzing
###          various aspects of the images, such as RNA intensity,
###          average intensity for regions, and pink pixel counting.
# ==================================================================

In [20]:
## Intensity Analysis for RNA
def get_intensity_of_RNA(rna_region):
    """
    Compute the average intensity of RNA (represented in the red channel) from a region.

    Parameters:
    - rna_region (ndarray): Image region (usually 3-channel) representing an RNA area.

    Returns:
    - float: Average intensity of the red channel for RNA pixels having an intensity greater than 2.
    """
    # Extract the red channel and create a boolean mask of pixels.
    RNA_red_pixels = rna_region[:, :, 2][rna_region[:, :, 2] > 2]
    return np.mean(RNA_red_pixels)


## Intensity Averaging Utility
def get_average_intensity_for_all_region(lst):
    """
    Compute the average of a list of intensities.

    Parameters:
    - lst (list): List of intensity values.

    Returns:
    - float: Average intensity. Returns 0 if the list is empty.
    """
    if len(lst) == 0:
        return 0
    else:
        return sum(lst) / len(lst)


## Pink Pixel tracking
def get_pink_count_from_region(pink_channel):
    """
    Count the number of pink pixels in a given region.

    Parameters:
    - pink_channel (ndarray): Image region to search for pink pixels.

    Returns:
    - int: Count of pink pixels in the region.
    """
    # Define the lower and upper bounds of the pink color range
    lower_pink = np.array([140, 0, 160])
    upper_pink = np.array([255, 100, 255])

    # Create a mask for pixels within the pink color range
    mask = cv2.inRange(pink_channel, lower_pink, upper_pink)

    return np.count_nonzero(mask)


# ==================================================================
# **RNA Data within Nucleus Analysis**
### **Summary**:
###          This section contains a function to extract and analyze
###          RNA data within the nucleus of a cell. It leverages
###          contours to define the nucleus region, then extracts RNA
###          data, pink pixel count, and more from this region.
# ==================================================================

In [25]:
## Function: Extract & Analyze RNA Data within Nucleus
def get_RNA_data_within_nucleus(mask_blue, contour, rna_image, nucleus_image, rgb_pink):
    """
    Extract and analyze RNA data within a specified nucleus region.

    Parameters:
    - mask_blue (ndarray): Binary mask highlighting the nucleus region.
    - contour (array): Contour defining the nucleus boundary.
    - rna_image (ndarray): Image representing RNA data.
    - nucleus_image (ndarray): Image of the nucleus.
    - rgb_pink (ndarray): RGB image highlighting pink regions (possibly overlapping areas).

    Returns:
    - tuple: Extracted RNA region, count of RNA pixels, extracted nucleus channel,
             extracted RNA red channel, average RNA red intensity, and count of pink pixels.
    """
    # Get Extract the corresponding region from the rna image based on nucleus
    mask_contour = np.zeros_like(mask_blue.copy())
    cv2.drawContours(mask_contour, [contour], 0, 255, thickness=cv2.FILLED)

    # Get rna area
    rna_region = cv2.bitwise_and(rna_image, rna_image, mask=mask_contour)
    nucleus_channel = cv2.bitwise_and(nucleus_image, nucleus_image, mask=mask_contour)
    pink_channel = cv2.bitwise_and(rgb_pink, rgb_pink, mask=mask_contour)

    pink_pixel_count = get_pink_count_from_region(pink_channel)

    # Count of available RNA intensity inside region
    RNA_pixels_count = np.count_nonzero(rna_region[:, :, 2] > 2)

    RNA_red_avg = get_intensity_of_RNA(rna_region) if RNA_pixels_count > 0 else 0.0

    return rna_region, RNA_pixels_count, nucleus_channel[:, :, 0], rna_region[:, :, 2], RNA_red_avg, pink_pixel_count


## Function: Calculate Percentage of RNA within Nucleus
def calculate_percentage_of_RNA_within_nucleus(RNA_data_count, nucleus_data_count):
    """
    Calculate the percentage of RNA within a specified nucleus region.

    Parameters:
    - RNA_data_count (int): Count of RNA data pixels within the nucleus region.
    - nucleus_data_count (int): Total pixel count of the nucleus region.

    Returns:
    - float: Percentage of RNA data within the nucleus (capped at 100%).
    """
    percentage_for_nucleus = (RNA_data_count / nucleus_data_count) * 100
    return min(percentage_for_nucleus, 100)


# ====================================================================
# **Nucleus Boundary Analysis**
### **Summary**:
###          This section contains functions to determine and analyze
###          nucleus boundaries, compute RNA presence within these
###          boundaries, and provide a visual representation.
# ====================================================================


In [42]:
## Function: Find and Analyze Nucleus Boundaries
def find_and_analyze_nucleus_boundaries(nucleus_image, rna_image, rgb_pink):
    """
    Find the boundaries of the nucleus in the provided image, calculate metrics
    regarding RNA presence within the nucleus, and visualize the findings.

    Parameters:
    - nucleus_image (ndarray): The image containing the nucleus data.
    - rna_image (ndarray): The image containing the RNA data.
    - rgb_pink (ndarray): The RGB image for pink detection.

    Returns:
    - tuple: A tuple containing:
        - Processed nucleus image.
        - Overall progress of RNA within the nucleus.
        - Overall intensity of RNA.
        - Overall progress using the pink region.
        - Count of valid nucleus regions.
    """

    imghsv = cv2.cvtColor(nucleus_image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the blue color range
    lower_blue = np.array([110, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # A mask for the nucleus
    mask_blue = cv2.inRange(imghsv, lower_blue, upper_blue)

    blue_contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    nucleus = np.copy(nucleus_image)
    rna = np.copy(rna_image)

    print("Total nucleus: ", len(blue_contours))

    rna_region_count = 0
    nucleus_region_count = 0
    pink_region_count = 0
    count = 0
    red_i_list = []

    for i, contour in enumerate(blue_contours):

      region_rna, RNA_data_count, nucleus_channel, rna_channel, RNA_I_avg, pink_pixel_count = get_RNA_data_within_nucleus(mask_blue, contour, rna_image, nucleus_image, rgb_pink)

      # get count of data in the nucleus
      nucleus_data_count = cv2.contourArea(contour)

      # check valid nucleus
      if nucleus_data_count > 0:

          x, y, w, h = cv2.boundingRect(contour)
          cv2.rectangle(nucleus, (x, y), (x + w, y + h), (0, 255, 0), 1)

          red_i_list.append(RNA_I_avg)

          rna_region_count += RNA_data_count
          nucleus_region_count += nucleus_data_count
          pink_region_count += pink_pixel_count

          percentage_for_nucleus = calculate_percentage_of_RNA_within_nucleus(RNA_data_count, nucleus_data_count)
          percentage_of_nucleus_using_pink = calculate_percentage_of_RNA_within_nucleus(pink_pixel_count, nucleus_data_count)

          # Print the percentage
          print(f"Percentage of RNA in nucleus {i + 1}: {percentage_for_nucleus:.2f}%")

          # Print the percentage using pink color
          print(f"Percentage of RNA in nucleus using pink color {i + 1}: {percentage_of_nucleus_using_pink:.2f}%")

          # generate correlation coefficient between the nucleus and RNA channels
          correlation_coefficient = np.corrcoef(nucleus_channel.flatten(), rna_channel.flatten())[0, 1]

          print(f"Correlation coefficient of nucleus {i + 1} & RNA is : {correlation_coefficient:.4f}")

          cv2.drawContours(nucleus, [contour], -1, (0, 255, 0), 1)
          count += 1
          # Write the red percentage on top of the contour
          cv2.putText(nucleus, f"{percentage_for_nucleus:.2f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    overall_progress = calculate_percentage_of_RNA_within_nucleus(rna_region_count, nucleus_region_count)

    overall_intensity = get_average_intensity_for_all_region(red_i_list)

    overall_progress_with_pink = calculate_percentage_of_RNA_within_nucleus(pink_region_count, nucleus_region_count)

    return nucleus, overall_progress, overall_intensity, overall_progress_with_pink, count

# ==========================================
# **Image Data Analysis Over Time**
### **Summary:**
###          This section processes and analyzes a series of images stored in
###          time sequenced folders to determine RNA interactions within the
###          nucleus.Metrics like progress over time, intensity, and
###          interactions using pink color from the RGB images are collected.
# ==========================================


In [ ]:
# Lists to store metrics over time
time_data = []
progress_data = []
progress_data_with_pink = []
intensity_data = []
nucleus_total = []

# Process each hour's data and images
for folder_name in subdirectories:

    tiff_file_path = os.path.join(data_dir, folder_name, 'montage.tif')

    if os.path.isfile(tiff_file_path):

        image = tiff.imread(tiff_file_path)
        opencv_frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        height, width, channels = opencv_frame.shape

        # Splitting the frame into 5 segments
        part_width = width // 5

        divided_images = []

        for i in range(5):
            start_col = i * part_width
            end_col = (i + 1) * part_width
            part_image = opencv_frame[:, start_col:end_col]

            divided_images.append(part_image)

        print(f"Processed parts from folder: {folder_name}")

        # perform optation on each hour data
        nucleus_boundary_img, overall_progress, overall_intensity, overall_progress_with_pink, nucleus_count = find_and_analyze_nucleus_boundaries(divided_images[3], divided_images[0], divided_images[-1])


        # show input data
        plt.subplot(2, 3, 1)
        plt.imshow(cv2.cvtColor(divided_images[3], cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Input nucleus data ')

        plt.subplot(2, 3, 2)
        plt.imshow(cv2.cvtColor(divided_images[0], cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Input RNA data')

        plt.subplot(2, 3, 3)
        plt.imshow(cv2.cvtColor(divided_images[-1], cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Input RGB data at {folder_name} hour')

        plt.tight_layout()
        plt.show()


        # convert OverallIntensity in scale of 100 from 255
        overall_intensity_in_percentage = (overall_intensity * 100) / 255


        # append time and OverallProgress to the lists
        time_data.append(folder_name)
        progress_data.append(overall_progress)
        intensity_data.append(overall_intensity_in_percentage)
        nucleus_total.append(nucleus_count)
        progress_data_with_pink.append(overall_progress_with_pink)

        print(f"Overall Progress at {folder_name} hour is {overall_progress:.2f} %")

        print(f"Overall Progress at {folder_name} hour is with RGB image (pink color){overall_progress_with_pink:.2f} %")


        # plt.subplot(1, num_frames, frame_idx + 1)  # Subplot for each frame
        plt.imshow(cv2.cvtColor(nucleus_boundary_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Data at {folder_name} hour, Progress {overall_progress:.2f} %')
        plt.show()


    else:
        print(f"TIFF file not found in folder: {folder_name}")


# ==========================================
# **RNA Progression Visualization**
### **Summary:**
###   This section focuses on plotting the RNA progression metrics over time.
###   Multiple lines represent different measures, and the data points are
###   annotated for clarity.
# ==========================================


In [ ]:
import matplotlib.pyplot as plt

# ==========================================
# Section: RNA Progression Visualization
# Summary: This section focuses on plotting the RNA progression metrics over time.
#          Multiple lines represent different measures, and the data points
#          are annotated for clarity.
# ==========================================

# Create a new figure with specified dimensions
plt.figure(figsize=(12, 6))

# Plot Overall Progress vs Time
plt.plot(time_data, progress_data, marker='o', linestyle='-', color='b', label='RNA Nucleus Interaction')

# Plot RNA Intensity vs Time
plt.plot(time_data, intensity_data, marker='s', linestyle='--', color='r', label='RNA intensity (red)')

# Plot Overall Progress with Pink vs Time
plt.plot(time_data, progress_data_with_pink, marker='^', linestyle=':', color='purple', label='Pink intensity (RGB Merged)')

# Setting various attributes for the plot
plt.xlabel('Time', fontsize=14)
plt.ylabel('Overall Progress (%)', fontsize=14)
plt.title('RNA progression vs. Time', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.legend(loc='best', fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=12)

# Highlight the region between 0% and 100%
plt.axhspan(0, 100, facecolor='lightgray', alpha=0.5)

# Annotate each data point
for i, progress in enumerate(progress_data):
    # Annotations for progress, intensity, and progress with pink data
    plt.annotate(f'{progress:.2f}%', (time_data[i], progress + 2), ha='center', fontsize=10)
    plt.annotate(f'{intensity_data[i]:.2f}%', (time_data[i], intensity_data[i] + 2), ha='center', fontsize=10)
    plt.annotate(f'{progress_data_with_pink[i]:.2f}%', (time_data[i], progress_data_with_pink[i] + 2), ha='center', fontsize=10)

    # Annotation for the total number of nuclei
    plt.annotate(f'Nucleus: {nucleus_total[i]}', (time_data[i], progress - 5), ha='center', fontsize=10, color='darkred')

# Show the plot
plt.tight_layout()
plt.show()


# ==========================================
# **Calculate Average RNA Intensity vs Time**
### **Summary:**
### This section focuses on visualizing the average RNA intensity over time
### using a bar chart. The RNA intensity values are plotted against specific
### time intervals, and annotations provide further information on each bar.
# ==========================================


In [ ]:
# create a plot of Avg. RNA intensity vs. Time

plt.figure(figsize=(12, 6))
plt.bar(time_data, intensity_data, color='b', alpha=0.6, label='RNA intensity')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Avg. RNA intensity', fontsize=14)
plt.title('Avg. RNA intensity vs. Time', fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()

for i, progress in enumerate(intensity_data):
    plt.annotate(f'{progress:.2f}', (time_data[i], progress + 4), ha='center', fontsize=10)
    plt.annotate(f'Nucleus: {nucleus_total[i]}', (time_data[i], progress - 8), ha='center', fontsize=10, color='darkred')

plt.legend(loc='best', fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.axhspan(0, max(intensity_data), facecolor='lightgray', alpha=0.5)

plt.show()


## Process with lsm file

### Function for Recognition of rna progression in each nucleus boundaries for lsm



In [11]:
def get_RNA_data_within_nucleus_lsm(mask_blue, contour, rna_image, nucleus_image):

    # Get Extract the corresponding region from the rna image based on nucleus
    mask_contour = np.zeros_like(mask_blue)
    cv2.drawContours(mask_contour, [contour], 0, 255, thickness=cv2.FILLED)

    # Get rna area
    rna_region = cv2.bitwise_and(rna_image, rna_image, mask=mask_contour)
    nucleus_channel = cv2.bitwise_and(nucleus_image, nucleus_image, mask=mask_contour)

    # Count of available RNA values inside region
    RNA_pixels_count = np.count_nonzero(rna_region[:, :, 2] > 5)

    return rna_region, RNA_pixels_count, nucleus_channel[:, :, 0], rna_region[:, :, 2]

In [12]:
def Get_Percentage_of_RNA_within_nucleus_lsm(RNA_data_count, nucleus_data_count):
    # calculate percentage of RNA within nucleus
    percentage_for_nucleus = (RNA_data_count / nucleus_data_count) * 100
    percentage_for_nucleus = min(percentage_for_nucleus, 100)
    return percentage_for_nucleus

### Function for Identify the boundaries for LSM

In [13]:
def find_nucleus_boundaries_lsm(nucleus_image, rna_image):

  min_width_threshold = 25

  imghsv = cv2.cvtColor(nucleus_image, cv2.COLOR_BGR2HSV)

  lower_blue = np.array([110, 50, 50])
  upper_blue = np.array([130, 255, 255])

  # A mask for the nucleus
  mask_blue = cv2.inRange(imghsv, lower_blue, upper_blue)

  blue_contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  count_nucleus = 0
  nucleus = np.copy(nucleus_image)

  rna_region_count = 0
  nucleus_region_count = 0


  for i, contour in enumerate(blue_contours):

    x, y, w, h = cv2.boundingRect(contour)

    # check valid nucleus
    if w >= min_width_threshold:

        region_rna, RNA_data_count, nucleus_channel, rna_channel = get_RNA_data_within_nucleus_lsm(mask_blue, contour, rna_image, nucleus_image)

        nucleus_data_count = cv2.contourArea(contour)

        rna_region_count += RNA_data_count
        nucleus_region_count += nucleus_data_count

        percentage_for_nucleus = Get_Percentage_of_RNA_within_nucleus_lsm(RNA_data_count, nucleus_data_count)

        # Print the percentage
        print(f"Percentage of RNA in nucleus {i + 1}: {percentage_for_nucleus:.2f}%")
        # generate correlation coefficient between the nucleus and RNA channels
        correlation_coefficient = np.corrcoef(nucleus_channel.flatten(), rna_channel.flatten())[0, 1]

        print(f"Correlation coefficient of nucleus {i + 1} & RNA is : {correlation_coefficient:.4f}")

        cv2.drawContours(nucleus, [contour], -1, (0, 255, 0), 2)
        cv2.rectangle(nucleus, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Write the red percentage on top of the contour
        cv2.putText(nucleus, f"{percentage_for_nucleus:.2f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 2)

        count_nucleus += 1

  OverallProgress = Get_Percentage_of_RNA_within_nucleus_lsm(rna_region_count, nucleus_region_count)

  return nucleus, count_nucleus, OverallProgress


### Process on each hour data

In [ ]:
# lists to store time and progress data
time_data = []
progress_data = []
nucleus_total = []

for folder_name in subdirectories:

    # Assuming rna file path 1.lsm first frame have rna details
    rna_file_path = os.path.join(data_dir, folder_name, '1.lsm')

    # Assuming nucleus file path 4.lsm 4th frame have nucleus details
    nucleus_file_path = os.path.join(data_dir, folder_name, '4.lsm')

    if os.path.isfile(nucleus_file_path):

        lsm_image = tiff.imread(nucleus_file_path)

        rna_file =  tiff.imread(rna_file_path)

        num_frames = len(lsm_image)
        # print(num_frames)

        # get nucleus frame
        nucleus_frame = cv2.cvtColor(lsm_image[-1], cv2.COLOR_RGB2BGR)

        # Split the last frame into its RGB channels
        b, g, r = cv2.split(nucleus_frame)

        b_channel = np.ones_like(b)
        g_channel = np.zeros_like(g)
        r_channel = np.zeros_like(r)

        # visual transformation nucleus
        nucleus_image = cv2.merge((b, g_channel, r_channel))

        print(f"Process at time / folder: {folder_name} hour")

        # get rna frame
        rna_frame = cv2.cvtColor(rna_file[0], cv2.COLOR_RGB2BGR)

        rna_b, rna_g, rna_r = cv2.split(rna_frame)

        b_channel = np.zeros_like(b)
        g_channel = np.zeros_like(g)
        r_channel = np.zeros_like(r)

        # visual transformation RNA
        rna_image = cv2.merge((b_channel, g_channel, rna_r))



        # perfrom operation on data
        nucleus_boundary_img, count_nucleus, OverallProgress = find_nucleus_boundaries_lsm(nucleus_image, rna_image)

        # view nucleus and RNA from lsm

        plt.subplot(2, 2, 1)
        plt.imshow(cv2.cvtColor(nucleus_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Input nucleus data at {folder_name} hour')

        plt.subplot(2, 2, 2)
        plt.imshow(cv2.cvtColor(rna_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Input RNA data at {folder_name} hour')

        plt.tight_layout()
        plt.show()



        print(f"Overall Progress at {folder_name} hour is {OverallProgress:.2f} %")

        print("Total nucleus: ", count_nucleus)

        # append time and OverallProgress to the lists
        time_data.append(folder_name)
        progress_data.append(OverallProgress)
        nucleus_total.append(count_nucleus)

        plt.imshow(cv2.cvtColor(nucleus_boundary_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Data at {folder_name} hour, Progress {OverallProgress:.2f} %')
        plt.show()

    else:
        print(f"Lsm file not found in folder: {folder_name}")


### RNA progression vs Time

In [ ]:
import matplotlib.pyplot as plt

# Create a plot of Overall Progress vs. Time
plt.figure(figsize=(12, 6))
plt.plot(time_data, progress_data, marker='o', linestyle='-', color='b', label='Progress')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Overall Progress (%)', fontsize=14)
plt.title('RNA progression vs. Time', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()


plt.legend(loc='best', fontsize=12)


plt.tick_params(axis='both', which='major', labelsize=12)

plt.axhspan(0, 100, facecolor='lightgray', alpha=0.5)

for i, progress in enumerate(progress_data):
    plt.annotate(f'{progress:.2f}%', (time_data[i], progress + 2), ha='center', fontsize=10)
    plt.annotate(f'Nucleus: {nucleus_total[i]}', (time_data[i], progress - 5), ha='center', fontsize=10, color='darkred')

plt.show()